In [8]:
import os, re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [9]:
# cd drive/MyDrive/Colab\ Notebooks/Theoretical_VTDP
DATA_DIR = "../data/Theoretical_VTDP"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Combining All Data Files into One

In [4]:
import os
import re

pattern = r"h(\d+)_flux(\d+)_abs(\d+).*?_surf(\d+).*?_(\d+)s"

# % mapping tables for quick lookup
h_map = {2:0.0375, 3:0.084, 6:0.1575}
flux_map = {88: 25900, 78: 21250, 73: 19400}
abs_val_map = {0: 3, 92: 100}
surf_map = {0:0.98, 1:0.76}

dataframes = []
for filename in os.listdir("./"):
    if filename.endswith(".csv"):

        match = re.search(pattern, filename)
        if match:
            h = int(match.group(1))
            flux = int(match.group(2))
            abs_val = int(match.group(3))
            surf = int(match.group(4))
            min_time = int(match.group(5))

            # flux from 73 → 19.4, 88 → 25.9
            if h in h_map:
              h = h_map[h]
            if flux in flux_map:
              flux = flux_map[flux]

            if abs_val in abs_val_map:
              abs_val = abs_val_map[abs_val]

            if surf in surf_map:
              surf = surf_map[surf]



            print(filename)
            file_path = os.path.join("", filename)
            print(file_path)
            data = pd.read_csv(file_path, encoding="utf-8-sig")  # UTF-8 with BOM

            if 'Time' in data.columns:
                data = data[data['Time'] >= min_time].copy()

            data["h"] = h
            data["flux"] = flux
            data["abs"] = abs_val
            data["surf"] = surf

            dataframes.append(data)

# Combine all dataframes into one
combined_data = pd.concat(dataframes, ignore_index=True)

print(combined_data.head())

combined_data.to_csv("PG_combined_data.csv", index=False)


h6_flux88_abs0_surf0_longRun_762s - Sheet1_processed.csv
h6_flux88_abs0_surf0_longRun_762s - Sheet1_processed.csv
h6_flux88_abs0_surf1_491s - Sheet3_processed.csv
h6_flux88_abs0_surf1_491s - Sheet3_processed.csv
h3_flux88_abs90_surf0_Redone_again_266s-Sheet1_processed.csv
h3_flux88_abs90_surf0_Redone_again_266s-Sheet1_processed.csv
h6_flux88_abs0_surf0_longRun_558s - Sheet2_processed.csv
h6_flux88_abs0_surf0_longRun_558s - Sheet2_processed.csv
h6_flux88_abs20_surf1_613s - Sheet1_processed.csv
h6_flux88_abs20_surf1_613s - Sheet1_processed.csv
h6_flux88_abs0_surf0_800s - Sheet2_processed.csv
h6_flux88_abs0_surf0_800s - Sheet2_processed.csv
h6_flux88_abs0_surf1_newSalt_689s - Sheet1_processed.csv
h6_flux88_abs0_surf1_newSalt_689s - Sheet1_processed.csv
h3_flux88_abs90_surf0_Redone_again3_344s-Sheet1_processed.csv
h3_flux88_abs90_surf0_Redone_again3_344s-Sheet1_processed.csv
h2_flux88_abs92_surf0_probeUp_193s - Sheet1_processed.csv
h2_flux88_abs92_surf0_probeUp_193s - Sheet1_processed.csv


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
# so for this model, the training data will be
# h (depth), flux (q0), abs (absorption coefficient k), surf (surface emisisivty epsilon), and varying time (tStep)
# and y will be the thermal couples values FOR THAT TIME
data = pd.read_csv("PG_combined_data.csv")

# removing these two columns, some data files have these, some don't. I'll worry about them later +
# need to ask what they do again, idk why there is a 11th temp
data.drop(columns=["TC_9_5", "TC_Bottom_rec_groove","TC_wall_ins_ext", "TC_bottom_ins_groove", "Theoretical_Temps_11"], axis=1, inplace=True)
depth_cols = [c for c in data.columns if c.startswith("Depth_")]


data.drop(columns=depth_cols, inplace=True)

# making sure no nulls in  other  rows after r emoving these two
print("Null Check: ", data.isnull().sum())

Theoretical_temps = [x for x in data.columns if x.startswith("Theoretical_Temps_")]
X = data[["Time", "h", "flux", "abs", "surf"] + Theoretical_temps]
print("INPUTS: \n", X.head())
y = data.drop(columns=["Time", "h", "flux", "abs", "surf"] + Theoretical_temps)
print("OUPUTS: \n", y.head())

y_columns = y.columns
# MinMaxScaler -> normalization, standardize when data is not normal (not in our case)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)




input_size = X_train.shape[1]
print("input size: ", input_size)

output_size = y_train.shape[1]
print("output size: ", output_size)

print("Amount of Training Data: ", X_train.shape[0])






Null Check:  Time                    0
TC1_tip                 0
TC2                     0
TC3                     0
TC4                     0
TC5                     0
TC6                     0
TC7                     0
TC8                     0
TC9                     0
TC10                    0
Theoretical_Temps_1     0
Theoretical_Temps_2     0
Theoretical_Temps_3     0
Theoretical_Temps_4     0
Theoretical_Temps_5     0
Theoretical_Temps_6     0
Theoretical_Temps_7     0
Theoretical_Temps_8     0
Theoretical_Temps_9     0
Theoretical_Temps_10    0
h                       0
flux                    0
abs                     0
surf                    0
dtype: int64
INPUTS: 
    Time       h   flux  abs  surf  Theoretical_Temps_1  Theoretical_Temps_2  \
0   762  0.1575  25900    3  0.98           313.462921           334.837384   
1   763  0.1575  25900    3  0.98           313.502765           334.881585   
2   764  0.1575  25900    3  0.98           313.542606           334.925783  

In [9]:
# basic neural net
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = NeuralNet(input_size, output_size)

# coommon loss func n optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 500
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad() # resetting gradients
    outputs = model(X_train)

    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}")

# no more calculating grad descent/back propgation, only forward
model.eval()
with torch.no_grad():
    y_pred = model(X_test).numpy()

# change back predictions and true values
y_pred_real = y_scaler.inverse_transform(y_pred)
y_test_real = y_scaler.inverse_transform(y_test.numpy())

# get RMSE per output
rmse_per_output = np.sqrt(np.mean((y_pred_real - y_test_real)**2, axis=0))
rmse_overall = np.mean(rmse_per_output)


print("\nRMSE per output (°C):")
for col, rmse in zip(y_columns, rmse_per_output):
    print(f"{col}: {rmse:.3f} °C")

print(f"\nAverage RMSE across all outputs: {rmse_overall:.3f} °C")

torch.save(model.state_dict(), "thermal_model_weights.pth")
print("Model weights saved to 'thermal_model_weights.pth'")

Epoch [50/500], Loss: 0.0405014343559742
Epoch [100/500], Loss: 0.022291723638772964
Epoch [150/500], Loss: 0.014255122281610966
Epoch [200/500], Loss: 0.011769085191190243
Epoch [250/500], Loss: 0.011114482767879963
Epoch [300/500], Loss: 0.010584376752376556
Epoch [350/500], Loss: 0.010056163184344769
Epoch [400/500], Loss: 0.009541229344904423
Epoch [450/500], Loss: 0.009085705503821373
Epoch [500/500], Loss: 0.008708038367331028

RMSE per output (°C):
TC1_tip: 33.577 °C
TC2: 33.319 °C
TC3: 34.047 °C
TC4: 36.990 °C
TC5: 35.857 °C
TC6: 34.020 °C
TC7: 33.630 °C
TC8: 34.467 °C
TC9: 36.514 °C
TC10: 38.978 °C

Average RMSE across all outputs: 35.140 °C
Model weights saved to 'thermal_model_weights.pth'


# Using a Trained Model
To use a trained NN, you need the weights as a .pt or .pth file AND the architecture of the NN

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


input_size = 15
output_size = y.shape[1]
model = NeuralNet(input_size, output_size)
model.load_state_dict(torch.load('thermal_model_weights.pth')) # load  weights
model.eval() # eval mode, no trianing


NeuralNet(
  (fc1): Linear(in_features=15, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [12]:
# time (tStep), h (y), flux (q0), abs (k), surf (epsilon),
new_data = [[1049.0, 0.1575, 25900, 3, 0.98]]
# still have to scale d ata
new_data = X_scaler.transform(new_data)
new_data_tensor = torch.tensor(new_data, dtype=torch.float32)



with torch.no_grad():
    predictions = model(new_data_tensor)

# reverse normalziation to see the acutal outputs
predictions = y_scaler.inverse_transform(predictions.numpy())
print("Predictions:", predictions)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: X has 5 features, but MinMaxScaler is expecting 15 features as input.